In [1]:
# Initialize
import librosa
import soundfile as sf
import numpy as np

In [2]:
# Basic example

# 1. Get the file path to an included audio example
disco_strings_file = 'test_samples/SC_DS_120_strings_stabs_swinging_upward_sting_Gmin.wav'
kshmr_guitar_file = 'test_samples/KSHMR_Latin_GTR_Guitar_90_Am.wav'

sample_rate = 44100 # TODO: determine best sample rate. 44.1 kHz looks most common

# TODO: figure out how to get things like pitch_shift to work with stereo,
# don't want to do all this processing in mono, unless it doesnt make a difference...
mono = True

# 2. Load the audio as a waveform `y`
#    Store the sampling rate as `sr`
current_sample, sample_rate = librosa.load(kshmr_guitar_file, sr=sample_rate, mono=mono)


### Grab BPM of sample
This process only applies to sample loops and not one-shots, since one-shots (typically) do not have a BPM. We are assuming the sample loop filename contains the BPM. The initial idea is to create a regex parser that just looks for any number between 50-200. 

This range is arbitrary, but it cannot be too low since many sample filenames contain a low number identifier. It cannot be too high since it could contain an instrument like 808. For example, in the sample, TL_Loops_Drums_Disco_Fill_03_108, we want to get the 108 BPM and ignore 03.

In [3]:
from sample_parser import parse_sample_bpm

# Basic example
disco_fill_sample = 'TL_Loops_Drums_Disco_Fill_03_108.wav'
sample_bpm = parse_sample_bpm(disco_fill_sample)
sample_bpm

108

In [4]:
'''
List of problematic sample filenames for BPM:
- FSW_song_starter_route69_c_minor_99.wav
- FSW_loop_melody_synth_oldlead_route69_99_c_minor.wav
- FSW_loop_bass_synth_ayayobass_route69_99_c_minor.wav
- 125bpm_vocal_98_fx.wav
- EVIGAN_drum_loop_80s_but_new_119.wav
- TA_MK3_FILL_ONESHOT_80_S_125.wav
- 
'''

# Problematic examples
song_starter_sample = 'EVIGAN_drum_loop_80s_but_new_119.wav'
sample_bpm = parse_sample_bpm(song_starter_sample)
sample_bpm

### Grab key of sample
This process applies to both sample loops and one-shots. We are assuming the sample filename contains the key. Most drums, risers, and other FX samples do not have a key in the filename, so we only grab for those that have them included.

The initial idea is to create a regex parser that looks for the key identifier. In Splice, it appears that all key identifiers are at least preceded by an underscore. This could be tricky, since I have seen a wide variety of formats. For example, E minor could look like any of these - Em, Emin, Eminor, E_m, E_min, E_minor, etc. Plus the variations having a sharp or flat in the name. This solution works for now but important to update if I find one more elegant and flexible.

In [5]:
# Basic example
from sample_parser import parse_sample_key

parse_sample_key(disco_strings_file)

<music21.key.Key of g minor>

### Grab all files in Splice directory
More description later. Mostly for testing

In [6]:
SPLICE_SAMPLES_PATH = '/Users/lukemainwaring/Splice/sounds/packs'

In [7]:
import glob
import os

# Easier to read without list comprehension
# splice_files = []
# splice_sample_files = glob.glob(SPLICE_SAMPLES_PATH + '/**/*.wav', recursive=True)
# for sample_file in splice_sample_files:
#     full_file_split = sample_file.split('/')
#     splice_files.append(full_file_split[-1])

splice_files = [sample_file.split('/')[-1] for sample_file in glob.glob(SPLICE_SAMPLES_PATH + '/**/*.wav', recursive=True)]
splice_files

['SPINNIN_synth_one_shot_high_brass_Bmaj.wav',
 'SPINNIN_synth_one_shot_classic_stab_Emaj.wav',
 'SPINNIN_synth_one_shot_short_piano_Fmin.wav',
 'SPINNIN_synth_one_shot_brass_hit_Emaj.wav',
 'SPINNIN_melody_loop_falling_bassline_126_Emin.wav',
 'SPINNIN_melody_loop_dirty_wubs_126_Fmin.wav',
 'SPINNIN_melody_loop_riddim_cowbell_lead_126_Fmin.wav',
 'SPINNIN_melody_loop_clubby_bassline_126_Fmin.wav',
 'SPINNIN_melody_loop_uk_bass_126_Emin.wav',
 'SPINNIN_melody_loop_warehouse_wobble_stab_125_Fmin.wav',
 'SPINNIN_melody_loop_intro_wubs_126_Cmaj.wav',
 'SPINNIN_melody_loop_cruisin_bass_128_Emin.wav',
 'SPINNIN_melody_loop_dirty_bass_126_Fmin.wav',
 'SPINNIN_melody_loop_fat_and_bouncy_126_Fmin.wav',
 'SPINNIN_drum_top_loop_dance_126.wav',
 'SPINNIN_clap_one_shot_thick.wav',
 'SPINNIN_kick_one_shot_punch.wav',
 'SPINNIN_drum_loop_deep_full_loop_126.wav',
 'SPINNIN_drum_fill_loop_gunshot_126.wav',
 'SPINNIN_vocal_one_shot_oooh.wav',
 'SPINNIN_uplifter_one_shot_cool.wav',
 'SC_RH_152_horns_cho

In [8]:
# Get full file paths for Splice samples for eventual audio processing
splice_files_path = glob.glob(SPLICE_SAMPLES_PATH + '/**/*.wav', recursive=True)
full_file_path_map = { sample : sample_path for sample, sample_path in zip(splice_files, splice_files_path)}

## Custom sample class
Every sample will be stored as one of our custom Sample objects, with the following required attributes:
- Name
- Tempo
- Key
- One shot
- Full path
- Instrument (leaving this one out for now)

In [9]:
from sample_processor import Sample

# Convert all Splice files to custom Sample object
sample_objects = []
for sample_file in splice_files:
    full_path = full_file_path_map[sample_file]
    new_sample = Sample(sample_file, full_path)
    sample_objects.append(new_sample)

In [10]:
len(sample_objects)

931

In [11]:
# Verify correct Sample object creation for loop sample
test_sample = sample_objects[10]

sample_output = (
    f'Name: {test_sample.name}\n'
    f'Tempo: {test_sample.tempo}\n'
    f'Key: {test_sample.key}\n'
    f'One Shot: {test_sample.one_shot}\n'
    f'Full path: {test_sample.full_path}'
)

print(sample_output)

Name: SPINNIN_melody_loop_intro_wubs_126_Cmaj.wav
Tempo: 126
Key: C major
One Shot: False
Full path: /Users/lukemainwaring/Splice/sounds/packs/Spinnin' Sounds Bass House Sample Pack/SPINNIN_BASS_HOUSE_sample_pack/SPINNIN_tonal/SPINNIN_melody_loops/SPINNIN_melody_loop_intro_wubs_126_Cmaj.wav


In [12]:
# Verify correct Sample object creation for one shot sample
test_sample_one_shot = sample_objects[20]

sample_output = (
    f'Name: {test_sample_one_shot.name}\n'
    f'Tempo: {test_sample_one_shot.tempo}\n'
    f'Key: {test_sample_one_shot.key}\n'
    f'One Shot: {test_sample_one_shot.one_shot}\n'
    f'Full path: {test_sample_one_shot.full_path}'
)

print(sample_output)

Name: SPINNIN_uplifter_one_shot_cool.wav
Tempo: None
Key: None
One Shot: True
Full path: /Users/lukemainwaring/Splice/sounds/packs/Spinnin' Sounds Bass House Sample Pack/SPINNIN_BASS_HOUSE_sample_pack/SPINNIN_fx/SPINNIN_sweeps/SPINNIN_uplifter_one_shot_cool.wav


In [13]:
# Verify correct Sample object creation for all Splice samples
for sample in sample_objects:
    sample_output = (
        f'Name: {sample.name}\n'
        f'Tempo: {sample.tempo}\n'
        f'Key: {sample.key}\n'
        f'One Shot: {sample.one_shot}\n'
        f'Full path: {sample.full_path}'
    )
    print(sample_output)
    print('--------------------------------------------------\n')

Name: SPINNIN_synth_one_shot_high_brass_Bmaj.wav
Tempo: None
Key: B major
One Shot: True
Full path: /Users/lukemainwaring/Splice/sounds/packs/Spinnin' Sounds Bass House Sample Pack/SPINNIN_BASS_HOUSE_sample_pack/SPINNIN_tonal/SPINNIN_synth_one_shots/SPINNIN_synth_one_shot_high_brass_Bmaj.wav
--------------------------------------------------

Name: SPINNIN_synth_one_shot_classic_stab_Emaj.wav
Tempo: None
Key: E major
One Shot: True
Full path: /Users/lukemainwaring/Splice/sounds/packs/Spinnin' Sounds Bass House Sample Pack/SPINNIN_BASS_HOUSE_sample_pack/SPINNIN_tonal/SPINNIN_synth_one_shots/SPINNIN_synth_one_shot_classic_stab_Emaj.wav
--------------------------------------------------

Name: SPINNIN_synth_one_shot_short_piano_Fmin.wav
Tempo: None
Key: f minor
One Shot: True
Full path: /Users/lukemainwaring/Splice/sounds/packs/Spinnin' Sounds Bass House Sample Pack/SPINNIN_BASS_HOUSE_sample_pack/SPINNIN_tonal/SPINNIN_synth_one_shots/SPINNIN_synth_one_shot_short_piano_Fmin.wav
-----------

Full path: /Users/lukemainwaring/Splice/sounds/packs/Serj Tankian Projectiles/SERJ_TANKIAN_sample_pack/SERJ_TANKIAN_tonal/SERJ_TANKIAN_piano/SERJ_TANKIAN_piano_loop_sorrowful_85_Gmin.wav
--------------------------------------------------

Name: SERJ_TANKIAN_piano_loop_building_100_Cmin.wav
Tempo: 100
Key: c minor
One Shot: False
Full path: /Users/lukemainwaring/Splice/sounds/packs/Serj Tankian Projectiles/SERJ_TANKIAN_sample_pack/SERJ_TANKIAN_tonal/SERJ_TANKIAN_piano/SERJ_TANKIAN_piano_loop_building_100_Cmin.wav
--------------------------------------------------

Name: SERJ_TANKIAN_bass_loop_ridem_100_Cmin.wav
Tempo: 100
Key: c minor
One Shot: False
Full path: /Users/lukemainwaring/Splice/sounds/packs/Serj Tankian Projectiles/SERJ_TANKIAN_sample_pack/SERJ_TANKIAN_tonal/SERJ_TANKIAN_guitar/SERJ_TANKIAN_electric_bass/SERJ_TANKIAN_bass_loop_ridem_100_Cmin.wav
--------------------------------------------------

Name: SERJ_TANKIAN_acoustic_guitar_loop_arabic_01_100_Cmin.wav
Tempo: 100
Key: 

In [14]:
# Problematic examples I could look into correcting
'''
Name: MAT_ZO_snare_094.wav
Tempo: 94
Key: None
One Shot: False
--------------------------------
***ALL SONNY_D SAMPLES SHOW UP AS D MAJOR
Name: SONNY_D_808_18_C.wav
Tempo: None
Key: D major
'''

'\nName: MAT_ZO_snare_094.wav\nTempo: 94\nKey: None\nOne Shot: False\n--------------------------------\n***ALL SONNY_D SAMPLES SHOW UP AS D MAJOR\nName: SONNY_D_808_18_C.wav\nTempo: None\nKey: D major\n'

### Transpose samples to the relevant key
The idea is to represent keys with the music21 library's key object. First, we get the range of keys that can be transposed without distorting the sample too much. I'm going with 2 for now, since quality sometimes sounds bad moving 3 semitones. For example, if a bass loop is already as low as it can get while still audible, we might not want to lower it 3 semitones. Maybe this range can be adjusted as part of the UX. Then, for each sample, we grab its key, and if it falls in the valid range then we transpose to the original song key.

In [15]:
from sample_transform import get_valid_key_range, get_candidate_sample_loops
from music21 import key

original_key = key.Key('C')
original_tempo = 120

In [16]:
# Creates set of keys within +/- 2 semitones
get_valid_key_range(original_key)

{<music21.key.Key of B- major>: -2,
 <music21.key.Key of B major>: -1,
 <music21.key.Key of C major>: 0,
 <music21.key.Key of D- major>: 1,
 <music21.key.Key of D major>: 2}

In [17]:
# We can also pass in a different range of steps
get_valid_key_range(original_key, 4)

{<music21.key.Key of A- major>: -4,
 <music21.key.Key of A major>: -3,
 <music21.key.Key of B- major>: -2,
 <music21.key.Key of B major>: -1,
 <music21.key.Key of C major>: 0,
 <music21.key.Key of D- major>: 1,
 <music21.key.Key of D major>: 2,
 <music21.key.Key of E- major>: 3,
 <music21.key.Key of E major>: 4}

In [18]:
# Get candidate sample loops with key in valid range and no key loops within valid bpm
# For example, this may include all samples as above plus drum loops, FX, risers, etc.
candidate_samples = get_candidate_sample_loops(sample_objects, original_tempo, original_key, require_key=False)
len(candidate_samples)

232

In [19]:
# Get candidate sample loops with key in valid range
candidate_samples = get_candidate_sample_loops(sample_objects, original_tempo, original_key)
len(candidate_samples)

14

### Generate transformations on candidate samples
This would be the step in which the candidate sample wav files get changed to match key/tempo of song

## Adjust all candidate samples to match song
If candidate samples have their bpm and key matched to the song, they can be evaluated in real time.

In [20]:
from sample_transform import match_sample

In [21]:
song_key = key.Key('C')
song_tempo = 120

for sample in candidate_samples:    
    matched_sample = match_sample(sample, song_key, song_tempo)
    sf.write('test_samples/candidate_samples_matched/' + sample.name, matched_sample, sample_rate)
    print('--------------------------------------\n')

Updating Tempo and Key for sample:  SPINNIN_melody_loop_intro_wubs_126_Cmaj.wav
Transpose steps:  0
Stretch factor:  0.9523809523809523
--------------------------------------

Updating Tempo and Key for sample:  SPINNIN_bass_synth_loop_wobble_mayhem_126_A#maj.wav
Transpose steps:  2
Stretch factor:  0.9523809523809523
--------------------------------------

Updating Tempo and Key for sample:  SPINNIN_drum_fill_loop_liftoff_126_C.wav
Transpose steps:  0
Stretch factor:  0.9523809523809523
--------------------------------------

Updating Tempo and Key for sample:  005_d__Drum_Beat_126bpm_-_AMPED_Zenhiser.wav
Transpose steps:  -2
Stretch factor:  0.9523809523809523
--------------------------------------

Updating Tempo and Key for sample:  015_b__Drum_Beat_126bpm_-_AMPED_Zenhiser.wav
Transpose steps:  1
Stretch factor:  0.9523809523809523
--------------------------------------

Updating Tempo and Key for sample:  TA_UHT_BASS_LOOP_BUNCE_126_C.wav
Transpose steps:  0
Stretch factor:  0.9523

### Standardize volume
Make sure sample is within similar volume range

In [22]:
# TODO: figure out if librosa, pydub, or wave is the best library for these steps
# move to sample_transform module when ready

### Standardize length
Make sure sample is looped to match the current song length

In [23]:
# TODO: move to sample_transform module when ready

In [24]:
# Basic example
guitar_sample, sample_rate = librosa.load(kshmr_guitar_file, sr=sample_rate, mono=mono)
looped_sample = np.append(guitar_sample, guitar_sample) # Looped once / doubles length

sf.write('test_samples/looped_sample.wav', looped_sample, sample_rate)

In [25]:
# TODO: write function that calculates how many times sample should be looped

# sample_length = len(current_sample)
# looped_sample = current_sample

# while sample_length < len(current_song):
#     looped_sample = np.append(looped_sample, current_sample)

### Combine sounds into one wav file
Now that the sample and current song are properly matched, combine and output as one file. This will be what the user hears, what gets inputted to the model, etc.

In [26]:
# Basic example
sample1, sample_rate = librosa.load(kshmr_guitar_file, sr=sample_rate, mono=mono)
sample2, sample_rate = librosa.load(disco_strings_file, sr=sample_rate, mono=mono)

combined_samples = sample1[:len(sample2)] + sample2 # These must be the same length
sf.write('test_samples/combined_samples.wav', combined_samples, sample_rate)